In [ ]:

import optuna
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

def objective(trial):
    alpha = trial.suggest_float("alpha", 0.01, 100.0, log=True)
    model = Ridge(alpha=alpha)
    score = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    return -score.mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Mejor alpha:", study.best_params['alpha'])
print("Mejor MSE:", study.best_value)



# Validación de Modelos: Train/Test Split, Validación Cruzada y Búsqueda de Hiperparámetros

Una parte esencial del desarrollo de modelos de machine learning es evaluar correctamente su rendimiento y generalización.

---

## ¿Por qué es importante la validación?

- Para estimar cómo funcionará el modelo en datos nuevos.
- Para evitar el sobreajuste.
- Para seleccionar los mejores hiperparámetros.

---

## Temas cubiertos:
1. Separación Train/Test
2. Validación Cruzada (K-Fold)
3. Búsqueda de Hiperparámetros (Grid Search y Random Search)


In [ ]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Cargar dataset de ejemplo
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X.shape, y.shape



## 1. Train/Test Split

La forma más simple de validar un modelo es dividir los datos en un conjunto de **entrenamiento** y otro de **prueba**.

- **Entrenamiento**: el modelo aprende.
- **Prueba**: evaluamos su capacidad de generalización.

```python
train_test_split(X, y, test_size=0.2, random_state=42)
```


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo = Ridge(alpha=1.0)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

print("MSE Test:", mean_squared_error(y_test, y_pred))



## 2. Validación Cruzada (Cross-Validation)

En lugar de una sola división, **K-Fold Cross-Validation** divide los datos en `k` bloques y entrena el modelo `k` veces, cada vez usando un bloque diferente como prueba.

### Beneficios:
- Usa más datos para entrenamiento.
- Proporciona estimaciones más robustas del desempeño del modelo.

```python
from sklearn.model_selection import cross_val_score
```


In [ ]:

from sklearn.model_selection import cross_val_score

modelo_cv = Ridge(alpha=1.0)
scores = cross_val_score(modelo_cv, X, y, cv=5, scoring='neg_mean_squared_error')
print("MSE promedio (5-fold CV):", -np.mean(scores))



## 3. Búsqueda de Hiperparámetros

La selección de hiperparámetros se puede automatizar mediante búsqueda en rejilla (**Grid Search**) o aleatoria (**Random Search**).

### Grid Search:
Prueba todas las combinaciones posibles de parámetros.

```python
from sklearn.model_selection import GridSearchCV
```

### Random Search:
Prueba un número fijo de combinaciones aleatorias.

```python
from sklearn.model_selection import RandomizedSearchCV
```


In [ ]:

from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X, y)

print("Mejor alpha:", grid.best_params_['alpha'])
print("Mejor MSE (CV):", -grid.best_score_)


In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {'alpha': uniform(0.01, 100)}
random_search = RandomizedSearchCV(Ridge(), param_distributions=param_dist, n_iter=10,
                                   cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X, y)

print("Alpha óptimo (Random Search):", random_search.best_params_['alpha'])
print("MSE óptimo (CV):", -random_search.best_score_)



## 4. Búsqueda Bayesiana de Hiperparámetros

La **búsqueda bayesiana** mejora el proceso de ajuste de hiperparámetros al modelar la función de puntuación como una función probabilística. En lugar de probar combinaciones al azar o exhaustivamente (como en Grid o Random Search), **aprende qué regiones del espacio de búsqueda tienen más probabilidades de contener buenos hiperparámetros** y explora en consecuencia.

Se basa en:
- Un modelo probabilístico (como Gaussian Process o Tree Parzen Estimator).
- Una estrategia de adquisición para decidir qué puntos explorar (como UCB, EI o PI).

### Ventajas:
- Más eficiente en problemas costosos.
- Encuentra buenos parámetros con menos evaluaciones.
- Escalable y extensible.

### Librerías útiles:
- `optuna`
- `scikit-optimize (skopt)`
- `bayesian-optimization`



## Ejercicio Final

1. Usa un modelo `Lasso` o `RandomForestRegressor`.
2. Aplica GridSearchCV para encontrar los mejores hiperparámetros.
3. Usa validación cruzada para comparar diferentes modelos y decidir el mejor.

¡Recuerda comparar MSE promedio y ajustar los parámetros correctamente!
